# Abstract Algebra

An implementation of finite algebras: Magma, Semigroup, Monoid, Group, Ring, Field.

For API documentation see: https://abstract-algebra.readthedocs.io/en/latest/index.html

In [1]:
import finite_algebras as alg
import json
import os

In [2]:
# Path to this repo
aa_path = os.path.join(os.getenv('PYPROJ'), 'abstract_algebra')

# Path to a directory containing Algebra definitions in JSON
alg_dir = os.path.join(aa_path, "Algebras")

## Groups

### Store Group in JSON format

In [3]:
# Path to a JSON file that defines the group V4
v4_json = os.path.join(alg_dir, "v4_klein_4_group.json")

!cat {v4_json}

{"name": "V4",
 "description": "Klein-4 group",
 "elements": ["e", "h", "v", "r"],
 "table": [[0, 1, 2, 3],
           [1, 0, 3, 2],
           [2, 3, 0, 1],
           [3, 2, 1, 0]]
}


### Read JSON Definition to Instantiate a Group Object

In [4]:
v4 = alg.make_finite_algebra(v4_json)
v4

Group(
V4,
Klein-4 group,
['e', 'h', 'v', 'r'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
)

The group can also be created using the Group constuctor.

In [5]:
alg.Group('V4',
          'Another way to construct V4',
          ['e',  'h',  'v', 'hv'],
          [[0, 1, 2, 3],
           [1, 0, 3, 2],
           [2, 3, 0, 1],
           [3, 2, 1, 0]]
         )

Group(
V4,
Another way to construct V4,
['e', 'h', 'v', 'hv'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
)

And the group can be created from a Python dictionary, but the more general algebra constructor, ``make_finite_algebra``, must be used:

In [6]:
v4_dict = {'name': 'V4',
           'description': 'Yet another way to define V4',
           'elements': ['e', 'h', 'v', 'hv'],
           'table': [[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]}

alg.make_finite_algebra(v4_dict)

Group(
V4,
Yet another way to define V4,
['e', 'h', 'v', 'hv'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
)

## Finding "About" an Algebra

The method, ``about``, will print information about a finite algebra.

By default, ``about`` prints the algebra's Cayley Table as it is represented internally using element indices.  It can be printed, instead, using element names by setting the argument, ``use_table_names``, to ``True``.

In [7]:
v4.about(use_table_names=True)


Group: V4
Description: Klein-4 group
Identity: e
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       h       h       2
      2       v       v       2
      3       r       r       2
Cayley Table (showing names):
[['e', 'h', 'v', 'r'],
 ['h', 'e', 'r', 'v'],
 ['v', 'r', 'e', 'h'],
 ['r', 'v', 'h', 'e']]


## Multiply Group Elements

Group multiplication operation takes zero or more arguments and returns the product according to the group's multiplication table (mult_table).

If no argument is provided, then the group's identity element is returned

In [8]:
v4.op()

'e'

If one argument is provided, then that argument is returned, assuming it's a valid element name.

In [9]:
v4.op('h')

'h'

If the one argument is not a valid element name, then an exception is raised.

In [10]:
try:
    v4.op('FOO')
except ValueError as err:
    print("Caught Error:")
    print(f"  {err}")

Caught Error:
  FOO is not a valid element name


If two or more arguments are provided, then their combined product is returned:

In [11]:
v4.op('h','v')  # h * v

'r'

In [12]:
v4.op('h', 'v', 'r')  # h * v * hv

'e'

### Inverse Elements

In [13]:
v4.inv('h')

'h'

That is, $h * h^{-1} = e$

In [14]:
v4.op('h', v4.inv('h'))

'e'

### Check if Abelian

In [15]:
v4.is_abelian()

True

### Check if Associative

NOTE: A group <u>must</u> be is_associative, so this check is done automatically when a group object is instantiated.

In [16]:
v4.is_associative()

True

### Group Generators

For now, there is only one group generator, for cyclic groups of any finite order:

In [17]:
z7 = alg.generate_cyclic_group(7)
z7.about()


Group: Z7
Description: Autogenerated cyclic group of order 7
Identity: e
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       a     a^6       7
      2     a^2     a^5       7
      3     a^3     a^4       7
      4     a^4     a^3       7
      5     a^5     a^2       7
      6     a^6       a       7
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6],
 [1, 2, 3, 4, 5, 6, 0],
 [2, 3, 4, 5, 6, 0, 1],
 [3, 4, 5, 6, 0, 1, 2],
 [4, 5, 6, 0, 1, 2, 3],
 [5, 6, 0, 1, 2, 3, 4],
 [6, 0, 1, 2, 3, 4, 5]]


### Derive Direct Product

The usual Python multiplication operator, ``*``, is overloaded for Groups to provide an easy method for computing Direct Products.  Element names are joined in Direct Products using the delimiter, ":", but that can be changed, if desired.

By default, the ``about`` method suppresses table output for Groups of order greater than 12, but than can be overridden, using ``max_size``, as shown below.

In [18]:
v4.direct_product_delimiter('-')  # Default delimiter is ':'
v4_x_v4 = v4 * v4
v4_x_v4.about(max_size=16)


Group: V4_x_V4
Description: Direct product of V4 & V4
Identity: e-e
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0     e-e     e-e       1
      1     e-h     e-h       2
      2     e-v     e-v       2
      3     e-r     e-r       2
      4     h-e     h-e       2
      5     h-h     h-h       2
      6     h-v     h-v       2
      7     h-r     h-r       2
      8     v-e     v-e       2
      9     v-h     v-h       2
     10     v-v     v-v       2
     11     v-r     v-r       2
     12     r-e     r-e       2
     13     r-h     r-h       2
     14     r-v     r-v       2
     15     r-r     r-r       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [1, 0, 3, 2, 5, 4, 7, 6, 9, 8, 11, 10, 13, 12, 15, 14],
 [2, 3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12, 13],
 [3, 2, 1, 0, 7, 6, 5, 4, 11, 10, 9, 8, 15, 14, 13, 12],
 [4, 5, 6, 7, 0, 1, 2, 3, 12, 13, 14, 15, 8, 9, 10, 11],
 [5, 4, 7, 6, 1, 0, 3, 2,

### Convert to Dictionary

In [19]:
v4.to_dict()

{'type': 'Group',
 'name': 'V4',
 'description': 'Klein-4 group',
 'elements': ['e', 'h', 'v', 'r'],
 'table': [[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]}

### Convert to JSON string

In [20]:
v4.dumps()

'{"type": "Group", "name": "V4", "description": "Klein-4 group", "elements": ["e", "h", "v", "r"], "table": [[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]}'

### Proper Subgroups

In [21]:
subs = v4.proper_subgroups()

print(f"\nAll {len(subs)} of these proper subgroups are, obviously, isomorphic to each other.\n")

for sub in subs:
    sub.about()
    print("\n" + "-"*50)


All 3 of these proper subgroups are, obviously, isomorphic to each other.


Group: V4_subgroup_0
Description: Subgroup of: Klein-4 group
Identity: e
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       v       v       2
Cayley Table (showing indices):
[[0, 1], [1, 0]]

--------------------------------------------------

Group: V4_subgroup_1
Description: Subgroup of: Klein-4 group
Identity: e
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       h       h       2
Cayley Table (showing indices):
[[0, 1], [1, 0]]

--------------------------------------------------

Group: V4_subgroup_2
Description: Subgroup of: Klein-4 group
Identity: e
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       r       r       2
Cayley Table (showing indices):
[[0, 1], [1, 0]]

---------------------------

### Resources

* <b>Book</b>: ["Visual Group Theory" by Nathan Carter](https://bookstore.ams.org/clrm-32)
* [Group Explorer](https://nathancarter.github.io/group-explorer/index.html) -- Visualization software for the abstract algebra classroom
* [Groupprops, The Group Properties Wiki (beta)](https://groupprops.subwiki.org/wiki/Main_Page)
* [GroupNames](https://people.maths.bris.ac.uk/~matyd/GroupNames/index.html) -- "A database, under construction, of names, extensions, properties and character tables of finite groups of small order."
* [GAP](https://www.gap-system.org/#:~:text=What%20is%20GAP%3F,data%20libraries%20of%20algebraic%20objects.) -- "Groups, Algorithms, Programming - a System for Computational Discrete Algebra"
* [Groups of small order](http://www.math.ucsd.edu/~atparris/small_groups.html): Compiled by John Pedersen, Dept of Mathematics, University of South Florida
* [List of small groups](https://en.wikipedia.org/wiki/List_of_small_groups): Finite groups of small order up to group isomorphism
* [Classification of Groups of Order n ≤ 8 (PDF)](http://www2.lawrence.edu/fast/corrys/Math300/8Groups.pdf)
* [Subgroups of Order 4 (PDF)](http://newton.uor.edu/facultyfolder/beery/abstract_algebra/08_SbgrpsOrder4.pdf)
* Klein four-group, V4
  * [Wikipedia](https://en.wikipedia.org/wiki/Klein_four-group)
  * [Group Explorer](https://github.com/nathancarter/group-explorer/blob/master/groups/V_4.group)
* Cyclic group
  * [Wikipedia](https://en.wikipedia.org/wiki/Cyclic_group)
  * [Z4, cyclic group of order 4](https://github.com/nathancarter/group-explorer/blob/master/groups/Z_4.group)
* Symmetric group
  * [Symmetric group on 3 letters](https://github.com/nathancarter/group-explorer/blob/master/groups/S_3.group). Another name for this group is <i>"Dihedral group on 3 vertices"</i>
* [Groupoids and Smarandache Groupoids](https://arxiv.org/ftp/math/papers/0304/0304490.pdf) by W. B. Vasantha Kandasamy